In [1]:
import jsonlines

In [2]:
def snli_preprocess(src_path, dst_path):
    all_data = {}
    with open(src_path, 'r', encoding='utf8') as f:
        for line in jsonlines.Reader(f):
            sent1 = line['sentence1']
            sent2 = line['sentence2']
            label = line['gold_label']
            if sent1 not in all_data:
                all_data[sent1] = {}
            if label == 'entailment':
                all_data[sent1]['entailment'] = sent2
            elif label == 'contradiction':
                all_data[sent1]['contradiction'] = sent2

                # example:{"origin": "在白云的蓝天下，一个孩子伸手摸着停在草地上的一架飞机的螺旋桨。", "entailment": "一个孩子正伸手摸飞机的螺旋桨。", "contradiction": "一个孩子在玩球。"}
    out_data = [{'origin': k,
                 # 对比学习数据对(用于有监督任务)
                 'entailment': v['entailment'],  # 正样本(x^+) 
                 'contradiction': v['contradiction']}  # 负样本(x^-)
                for k, v in all_data.items()
                if v.get('entailment') and v.get('contradiction')]

    with jsonlines.open(dst_path, 'w') as writer:
        writer.write_all(out_data)

In [3]:
snli_preprocess('cnsd-snli/cnsd_snli_v1.0.train.jsonl', 'cnsd-snli-process/train.jsonl')